<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/ways_of_convert_rubert_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

Этот ноутбук для бекендеров, которым дали модельку и сказали деплоить так, чтобы она быстро работала. И больше ничего не дали, кроме модельки
	
  (・_・ヾ

Устанавливаем нужные зависимости

In [4]:
!pip install onnx transformers onnxruntime folium==0.2.1 optimum[onnxruntime] nn_pruning

Фиксируем версии библиотек

In [37]:
!pip freeze > req.txt

Импорты

In [7]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from torch.nn.utils import prune
from optimum.onnxruntime import ORTQuantizer
from nn_pruning.inference_model_patcher import optimize_model

Качаем датасет, на котором будем проверять качество модели

In [8]:
!wget https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv

--2022-04-16 18:39:21--  https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv [following]
--2022-04-16 18:39:21--  https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21781685 (21M) [text/plain]
Saving to: ‘women-clothing-accessories.3-class.balanced.csv’

women-clothing-acce 100%[===================>]  20.77M  89.9MB/s    i

# Об нейросети

В качестве подопытного будем использовать [rubert-base-cased-sentiment](https://huggingface.co/blanchefort/rubert-base-cased-sentiment) для классификации русских предложений. Данная нейросеть предсказывает 3 метки класса, в зависимости от тона предложения - позитивное, негативное или нейтральное

Запускаем нейросеть как есть

In [82]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [10]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

Проверим время инференса модели

In [11]:
%%time
predict(text)

CPU times: user 121 ms, sys: 127 ms, total: 248 ms
Wall time: 460 ms


2

Проверим качество модели. Для проверки качества будем использовать один из датасетов, на котором обучалась модель, а именно [этот](https://github.com/sismetanin/rureviews)

In [12]:
df = pd.read_csv('/content/women-clothing-accessories.3-class.balanced.csv', delimiter='\t')
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


Для удобства немного изменим датасет - заменим метки класса на цифровые значения и выберем 1000 рандомных строк

In [13]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:1000]
mapping = {'negative': 2, 'positive': 1, 'neautral':0}
df = df.replace({'sentiment': mapping})
df.head()

,review,sentiment
0,размер s брала...выглядит как м,2
1,"Где мой заказ??? Я не получила платье, а в отс...",2
2,"самые обыкновенные носки,пришли быстро!",0
3,"pants are cut bad. one leg already, the other ...",2
4,"пришел пустой пакет,. в нем только резинка",2


Считаем качество

In [14]:
texts = list(df['review'])
labels = list(df['sentiment'])

In [15]:
predictions = [predict(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7758420684835778, recall: 0.7625989008405051, f1score: 0.7586448839591


Сохраним оригинальную модель и посмотрим на ее вес

In [89]:
torch.save(model, 'output/original.pt')

In [90]:
!du -shc output/original.pt

922M	output/original.pt
922M	total


# ONNX

Формат Open Neural Network Exchange (ONNX) обеспечит общий способ представления данных, используемых в нейронных сетях. Большинство платформ имеют сегодня собственный специфический формат моделей, которые способны работать с моделями других платформ только при использовании специальных инструментов преобразования форматов.

ONNX позволит осуществлять свободный обмен информацией, которой обладают модели, без процедуры преобразования. Модель, обученную на одной платформе, можно будет использовать и на другой платформе. Также можно будет модель, обученную на одном фреймворке, перенести на другой фреймворк.

Перевести модель в ONNX можно несколькими способами:

1. Есть библиотека transforms, [где все почти из коробки](https://huggingface.co/docs/transformers/serialization)

In [16]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [17]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [18]:
!mkdir -p output/onnx_transforms

In [19]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model,
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert-base-cased-sentiment.onnx"),
        opset=11)

Пробуем запустить в ONNX и посмотреть время инференса

In [20]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment.onnx", sess_options, providers)

In [21]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  outputs  = model_ONNX.run(None, dict(inputs))[0][0]
  result = np.where(outputs == np.amax(outputs))[0][0]
  return result

In [22]:
%%time
predict_onnx(text)

CPU times: user 79.4 ms, sys: 17 µs, total: 79.4 ms
Wall time: 79.2 ms


2

Считаем качество

In [23]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7758420684835778, recall: 0.7625989008405051, f1score: 0.7586448839591


Посмотрим на вес модели

In [24]:
!du -shc output/onnx_transforms/*

679M	output/onnx_transforms/rubert-base-cased-sentiment.onnx
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала на порядок выше, метрики качества не изменились.Вес модели уменьшился на 300 мб

# TorchScript

TorchScript — инструмент, который позволяет с помощью пары строк кода и нескольких щелчков мыши сделать из пайплайна на питоне отчуждаемое решение, которое можно встроить в систему на C++. А еще она будет на python работать быстрее из-за jit компиляции. В библиотеке transformers так же [есть почти из коробки](https://huggingface.co/docs/transformers/serialization#torchscript)

Первым шагом нужно перезагрузить модель, добавив флаг torchscript=True

In [25]:
model_torchscript = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, torchscript=True)

In [26]:
inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
traced_model = torch.jit.trace(model_torchscript, [torch.tensor(x) for x in inputs.values()])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [27]:
!mkdir -p output/torchscript

In [28]:
torch.jit.save(traced_model, "output/torchscript/rubert-base-cased-sentiment_traced.pt")

Пробуем предиктить

In [29]:
@torch.no_grad()
def predict_torchscript(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = traced_model(**inputs)[0]
    predicted = torch.nn.functional.softmax(outputs, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [66]:
%%time
predict_torchscript(text)

CPU times: user 110 ms, sys: 1.92 ms, total: 111 ms
Wall time: 116 ms


2

Считаем качество модели

In [31]:
predictions = [predict_torchscript(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7758420684835778, recall: 0.7625989008405051, f1score: 0.7586448839591


 Посмотрим на вес модели

In [32]:
!du -shc output/torchscript/*

679M	output/torchscript/rubert-base-cased-sentiment_traced.pt
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала немного выше, метрики качества не изменились.Вес модели изменился на 300мб примерно. Лучше, чем ничего

# Прунинг модели

Model Pruning — обрезание избыточных частей сети для ускорения инференса без потери точности. Наглядно — откуда, сколько и как можно вырезать.

Есть два варианта, как прунить модель.

1 вариант - библиотека [nn_pruning](https://github.com/huggingface/nn_pruning) от HuggingFace

2 вариант - делать через torch.nn.utils.prune. В качестве примера есть данный [ноутбук](https://github.com/Huffon/nlp-various-tutorials/blob/master/pruning-bert.ipynb)

[Ссылка](https://aclanthology.org/2020.repl4nlp-1.18.pdf) на почитать про прунинг модели BERT

Запруним encoder слои

In [83]:
pruned_model = model

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (pruned_model.bert.encoder.layer[i].attention.self.key, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.query, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

Выведем, что получилось

In [84]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0)

    denominator += pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 18.59%
Sparsity in Layer 1-th query weightt: 18.67%
Sparsity in Layer 1-th value weight: 26.79%

Sparsity in Layer 2-th key weight: 18.77%
Sparsity in Layer 2-th query weightt: 18.33%
Sparsity in Layer 2-th value weight: 25.69%

Sparsity in Layer 3-th key weight: 20.08%
Sparsity in Layer 3-th query weightt: 19.58%
Sparsity in Layer 3-th value weight: 23.53%

Sparsity in Layer 4-th key weight: 18.77%
Sparsity in Layer 4-th query weightt: 18.49%
Sparsity in Layer 4-th value weight: 24.32%

Sparsity in Layer 5-th key weight: 18.40%
Sparsity in Layer 5-th query weightt: 18.36%
Sparsity in Layer 5-th value weight: 23.00%

Sparsity in Layer 6-th key weight: 18.32%
Sparsity in Layer 6-th query weightt: 17.84%
Sparsity in Layer 6-th value weight: 21.55%

Sparsity in Layer 7-th key weight: 18.54%
Sparsity in Layer 7-th query weightt: 18.06%
Sparsity in Layer 7-th value weight: 22.07%

Sparsity in Layer 8-th key weight: 18.60%
Sparsity in Layer 8-th query weigh

Предиктим на запруненной модели

In [85]:
@torch.no_grad()
def predict_pruned(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = pruned_model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [86]:
%%time
predict_pruned(text)

CPU times: user 148 ms, sys: 8 ms, total: 156 ms
Wall time: 155 ms


2

In [61]:
!mkdir -p output/pruning

In [87]:
torch.save(pruned_model, 'output/pruning/rubert-base-cased-sentiment_pruned.pt')

Посчитаем качество модели

In [70]:
predictions = [predict_pruned(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7673895104392958, recall: 0.759069038827347, f1score: 0.7562227505683002


 Посмотрим на вес модели

In [88]:
!du -shc output/pruning/*

922M	output/pruning/rubert-base-cased-sentiment_pruned.pt
922M	total


Вывод - вес модели не поменялся, качество упало чуть-чуть по сравнению с оригинальной моделью, но не критично. Модель стала работать быстрее, но ценой небольшой потери качества

# Квантизация

Квантизация означает уменьшение численной точности весов модели. Один из популярных методов — k-means квантизация. Имея веса модели в матрице W с десятичными числами, веса кластеризуются с помощью k-means в N кластеров. Затем матрица W трансформируется в матрицу целых чисел от 1 до N, каждое из которых является указателем к центру кластера. Так можно сжать каждый элемент изначальной матрицы из 32-битного десятичного числа в log(N)-битные целые числа.

Есть три вида квантизации - статическая, динамическая и Quantization-Aware-Training(QAT)


Делаем тоже по лучшим гайдам https://github.com/huggingface/optimum



Динамическая квантизация не требует ничего, поэтому она самая простая

In [91]:
!mkdir -p output/quantization

In [92]:
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained("blanchefort/rubert-base-cased-sentiment", feature="sequence-classification")

# Quantize the model!
quantizer.export(
    onnx_model_path="output/quantization/rubert-base-cased-sentiment.onnx",
    onnx_quantized_model_output_path="output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx')

Пробуем запустить динамечески квантизированную ONNX модель и посмотреть на время инференса

In [94]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx", sess_options, providers)

In [100]:
%%time
predict_onnx(text)

CPU times: user 55.9 ms, sys: 856 µs, total: 56.7 ms
Wall time: 57.8 ms


2

Считаем качество

In [101]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7634891164434495, recall: 0.7515760349283417, f1score: 0.7473241689008997


Посмотрим на вес модели

In [ ]:
!du -shc output/quantization/*

679M	output/onnx_transforms/rubert-base-cased-sentiment.onnx
679M	total


Вывод:  по сравнению с моделью в формате ONNX скорость инференса модели стала  выше, метрики качества немного просели.Вес модели не уменьшился